<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/OrigAdultInflightResnet1410_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access/save files
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Define paths to training data
train_data_root = "/content/drive/My Drive/FYP/Dataset/Original_Adult_In-flight/train"
IMAGE_SHAPE = (224, 224)  # Image size for ResNet50
BATCH_SIZE = 32  # Larger batch size for faster training
EPOCHS = 20  # Initial number of epochs

# Output paths for saving models and other results
MODEL_SAVE_PATH = '/content/drive/My Drive/FYP/MODELS/best_resnet50_model_v2.keras'
CLASS_INDICES_SAVE_PATH = '/content/drive/My Drive/FYP/class_indices_v2.npy'

# Create ImageDataGenerator for training and validation with increased augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the pixel values
    validation_split=0.3,  # 30% of data used for validation
    rotation_range=20,  # Increase rotation for more variability
    zoom_range=0.2,  # Increase zoom for more variability
    width_shift_range=0.2,  # Add horizontal shift
    height_shift_range=0.2,  # Add vertical shift
    shear_range=0.2,  # Shear transformation
    brightness_range=[0.8, 1.2],  # Vary brightness
    horizontal_flip=True  # Randomly flip images horizontally
)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_data_root,
    subset="training",
    target_size=IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Validation generator
valid_generator = train_datagen.flow_from_directory(
    train_data_root,
    subset="validation",
    target_size=IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Save class indices mapping for future use
class_indices = train_generator.class_indices
np.save(CLASS_INDICES_SAVE_PATH, class_indices)
print(f"Class indices saved: {class_indices}")

# Load pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=IMAGE_SHAPE + (3,))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global pooling layer
x = Dense(1024, activation='relu')(x)  # Fully connected layer with 1024 units
predictions = Dense(len(class_indices), activation='softmax')(x)  # Output layer for classification

# Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the ResNet50 base layers initially (only train new layers)
# Unfreeze more layers for fine-tuning
for layer in base_model.layers[-100:]:  # Unfreeze more layers
    layer.trainable = True

# Re-compile the model with a moderate learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Model compiled successfully.")

# Early stopping to avoid overfitting, with increased patience
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model checkpoint to save the best model based on validation accuracy
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_loss', mode='min')

# List of callbacks
callbacks = [early_stopping, model_checkpoint]

# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

# Save the final model after training
final_model_path = '/content/drive/My Drive/FYP/MODELS/resnet50_final_model_v2.keras'
model.save(final_model_path)
print(f"Final model saved at: {final_model_path}")

# Unfreeze more layers for fine-tuning (e.g., last 50 layers)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Re-compile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,  # Fine-tune for more epochs
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

# Save the fine-tuned model
fine_tuned_model_path = '/content/drive/My Drive/FYP/MODELS/resnet50_fine_tuned_model_v2.keras'
model.save(fine_tuned_model_path)
print(f"Fine-tuned model saved at: {fine_tuned_model_path}")

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(valid_generator, steps=valid_generator.samples // BATCH_SIZE)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


Mounted at /content/drive
Found 176 images belonging to 2 classes.
Found 75 images belonging to 2 classes.
Class indices saved: {'Glaucous_Winged_Gull': 0, 'Slaty_Backed_Gull': 1}
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model compiled successfully.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 215s 35s/step - accuracy: 0.5918 - loss: 0.7175 - val_accuracy: 0.4219 - val_loss: 0.7996
Epoch 2/20
1/5 ━━━━━━━━━━━━━━━━━━━━ 1:51 28s/step - accuracy: 0.8750 - loss: 0.3262

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.8750 - loss: 0.3262 - val_accuracy: 1.0000 - val_loss: 0.4485
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 127s 27s/step - accuracy: 0.9103 - loss: 0.2065 - val_accuracy: 0.4219 - val_loss: 0.7049
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 473ms/step - accuracy: 1.0000 - loss: 0.0542 - val_accuracy: 1.0000 - val_loss: 0.6435
Epoch 5/20
